Check cuhk03 dataset:

# NOTE: we don't need this dataset for a while

Reference: [link](http://www.ee.cuhk.edu.hk/~xgwang/CUHK_identification.html)


Splits:
- classic: 1367/100 split
- new: 767/700 split (need to download from [link](https://github.com/zhunzhong07/person-re-ranking/tree/master/evaluation/data/CUHK03))
- There are 20 splits

Statistics:
- 1360 identities
- 13164 images
- 6 cameras

There is a "new" protocal that extends the dataset.

Arguments:
- labeled vs detected
    - manually labeled vs automatic detection (DPM)
- split number (20)

In [1]:
%cd ..

/home/ubuntu/dev/reid/pepper


In [2]:
# builtin
from collections import defaultdict
import os.path as osp
import re

# third-party
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

import h5py
from imageio import imsave
from scipy.io import loadmat

# mm
import mmcv

In [4]:
ROOT = 'data/cuhk03'

# image file (in MATLAB format)
data_dir = osp.join(ROOT, 'cuhk03_release')
raw_mat_path = osp.join(data_dir, 'cuhk-03.mat')

# new splits
split_new_det_mat_path = osp.join(ROOT, 'cuhk03_new_protocol_config_detected.mat')
split_new_lab_mat_path = osp.join(ROOT, 'cuhk03_new_protocol_config_labeled.mat')

assert osp.exists(data_dir)
assert osp.exists(raw_mat_path)
assert osp.exists(split_new_det_mat_path)
assert osp.exists(split_new_lab_mat_path)

# save paths! (don't care for now)
# new save path (not going to use here...)
# imgs_detected_dir = osp.join(ROOT, "images_detected")
# imgs_labeled_dir = osp.join(ROOT, "images_labeled")

# # classic save splits as json
# split_classic_det_json_path = osp.join(ROOT, "splits_classic_detected.json")
# split_classic_lab_json_path = osp.join(ROOT, "splits_classic_labeled.json")

# # new save splits as json
# split_new_det_json_path = osp.join(ROOT, "splits_new_detected.json")
# split_new_lab_json_path = osp.join(ROOT, "splits_new_labeled.json")

In [6]:
def process_images(img_refs, campid, pid, save_dir):
    img_paths = []  # Note: some persons only have images for one view
    for imgid, img_ref in enumerate(img_refs):
        img = _deref(img_ref)
        if img.size == 0 or img.ndim < 3:
            continue  # skip empty cell
        # images are saved with the following format, index-1 (ensure uniqueness)
        # campid: index of camera pair (1-5)
        # pid: index of person in 'campid'-th camera pair
        # viewid: index of view, {1, 2}
        # imgid: index of image, (1-10)
        viewid = 1 if imgid < 5 else 2
        img_name = "{:01d}_{:03d}_{:01d}_{:02d}.png".format(
            campid + 1, pid + 1, viewid, imgid + 1
        )
        img_path = osp.join(save_dir, img_name)
        if not osp.isfile(img_path):
            imsave(img_path, img)
        img_paths.append(img_path)
    return img_paths

def extract_img(image_type, imgs_dir):
    print("Processing {} images ...".format(image_type))
    meta_data = []
    for campid, camp_ref in enumerate(mat[image_type][0]):
        camp = _deref(camp_ref)
        num_pids = camp.shape[0]
        for pid in range(num_pids):
            img_paths = process_images(
                camp[pid, :], campid, pid, imgs_dir
            )
            assert (
                len(img_paths) > 0
            ), "campid{}-pid{} has no images".format(campid, pid)
            meta_data.append((campid + 1, pid + 1, img_paths))
        print(
            "- done camera pair {} with {} identities".format(
                campid + 1, num_pids
            )
        )
    return meta_data

In [5]:
mat = h5py.File(raw_mat_path, "r")

In [7]:
meta_detected = extract_img("detected", "images_dir)
meta_labeled = extract_img("labeled")

Processing detected images ...


NameError: name 'self' is not defined